<h1><font color = 'red'> <center>CVEN 5370 GIS Apps in Engineering: Assignment 4</h1>
<h3><font color = 'blue'> <center>Shaseevarajan Sivanantharajah</h3>

<h3>Problem Statement

The goal is to create a logistic regression model for predicting bridge condition (susceptibility to failure). We shall use Age, ADT, Material, Reconstruct as our **independant variable**. We shall recode the lowest bridge rating as 0 (satisfactory) and 1 (unsatisfactory) using the following scale rating between [0-5] (1 or unsatisfactory) otherwise satisfactory.

**WORK FLOW**
1. Read the 2024 data
2. Create a latitudeDD and longitudeDD for plotting
3. Create a FID (lat_long)
4. Clean or remove duplicates
5. Recode the lowest ratings as binary variables
6. Compute age
7. Extract the required variables into a new dataframe
8. Perform train-test split
9. Fit the model on the training data
10. Evaluate the model on the training data
11. Read the 2025 data
12. Create a new FID using the same concept as in step 3
13. Extract unique bridges in 2025
14. Make prediction using the model fit in step 9 and compare
15. Make plots and maps for visualization and analysis

**Code**

In [7]:
!pip install statsmodels

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 28.2 MB/s eta 0:00:00m eta 0:00:01:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.9/232.9 KB 61.8 MB/s eta 0:00:00


In [16]:
import os
import pandas as pd
import numpy as np
import geopandas as gpd
from matplotlib import pyplot as plt
import statsmodels.api as sm


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/vu1/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/vu1/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/vu1/.local/lib/python3.10/site-packages/ipy

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [17]:
path = "/home/vu1/Shaseevarajan Sivanantharajah/Assignment4"
os.chdir(path)

In [18]:
fname = "2024Data.csv"
a = pd.read_csv(fname, low_memory = False)
a.head()

,STATE_CODE_001,STRUCTURE_NUMBER_008,RECORD_TYPE_005A,ROUTE_PREFIX_005B,SERVICE_LEVEL_005C,ROUTE_NUMBER_005D,DIRECTION_005E,HIGHWAY_DISTRICT_002,COUNTY_CODE_003,PLACE_CODE_004,...,BRIDGE_LEN_IND_112,SCOUR_CRITICAL_113,FUTURE_ADT_114,YEAR_OF_FUTURE_ADT_115,MIN_NAV_CLR_MT_116,FED_AGENCY,SUBMITTED_BY,BRIDGE_CONDITION,LOWEST_RATING,DECK_AREA
0,48,0000000L2015318,1,6,4,00000,0,0,375,0,...,Y,7,200,2035,NaN,Y,68,F,6,169.84
1,48,000021521-00101,1,6,0,00000,0,2,71,0,...,Y,8,1,2042,NaN,Y,63,F,6,150.72
2,48,000021521-00181,1,6,0,00000,0,2,71,0,...,Y,8,1,2042,NaN,Y,63,G,7,74.40
3,48,000021521-TMP20,1,6,1,00000,0,2,71,0,...,Y,8,150,2042,NaN,Y,63,G,7,91.91
4,48,000021525-00012,1,6,0,00000,0,2,245,0,...,Y,8,120,2042,NaN,Y,63,P,4,204.30


In [19]:
a.columns

Index(['STATE_CODE_001', 'STRUCTURE_NUMBER_008', 'RECORD_TYPE_005A',
       'ROUTE_PREFIX_005B', 'SERVICE_LEVEL_005C', 'ROUTE_NUMBER_005D',
       'DIRECTION_005E', 'HIGHWAY_DISTRICT_002', 'COUNTY_CODE_003',
       'PLACE_CODE_004',
       ...
       'BRIDGE_LEN_IND_112', 'SCOUR_CRITICAL_113', 'FUTURE_ADT_114',
       'YEAR_OF_FUTURE_ADT_115', 'MIN_NAV_CLR_MT_116', 'FED_AGENCY',
       'SUBMITTED_BY', 'BRIDGE_CONDITION', 'LOWEST_RATING', 'DECK_AREA'],
      dtype='object', length=123)

In [22]:
a['FID']= a.LAT_016.astype(str)+"_"+a.LONG_017.astype(str)
a['FID']

0        35185702_101560762
1         29362100_94272880
2         29362940_94254920
3         29365160_94324020
4         29413300_94044860
                ...        
56724    31495329_106183492
56725     32430233_96561253
56726     32424987_96560305
56727     32424560_96561340
56728     29283700_98252680
Name: FID, Length: 56729, dtype: object

In [23]:
a.head()

,STATE_CODE_001,STRUCTURE_NUMBER_008,RECORD_TYPE_005A,ROUTE_PREFIX_005B,SERVICE_LEVEL_005C,ROUTE_NUMBER_005D,DIRECTION_005E,HIGHWAY_DISTRICT_002,COUNTY_CODE_003,PLACE_CODE_004,...,SCOUR_CRITICAL_113,FUTURE_ADT_114,YEAR_OF_FUTURE_ADT_115,MIN_NAV_CLR_MT_116,FED_AGENCY,SUBMITTED_BY,BRIDGE_CONDITION,LOWEST_RATING,DECK_AREA,FID
0,48,0000000L2015318,1,6,4,00000,0,0,375,0,...,7,200,2035,NaN,Y,68,F,6,169.84,35185702_101560762
1,48,000021521-00101,1,6,0,00000,0,2,71,0,...,8,1,2042,NaN,Y,63,F,6,150.72,29362100_94272880
2,48,000021521-00181,1,6,0,00000,0,2,71,0,...,8,1,2042,NaN,Y,63,G,7,74.40,29362940_94254920
3,48,000021521-TMP20,1,6,1,00000,0,2,71,0,...,8,150,2042,NaN,Y,63,G,7,91.91,29365160_94324020
4,48,000021525-00012,1,6,0,00000,0,2,245,0,...,8,120,2042,NaN,Y,63,P,4,204.30,29413300_94044860


In [32]:
aa = a[~a.duplicated(subset=['FID'], keep = False)]

In [33]:
len(aa)

56571

In [34]:
aa.head()

,STATE_CODE_001,STRUCTURE_NUMBER_008,RECORD_TYPE_005A,ROUTE_PREFIX_005B,SERVICE_LEVEL_005C,ROUTE_NUMBER_005D,DIRECTION_005E,HIGHWAY_DISTRICT_002,COUNTY_CODE_003,PLACE_CODE_004,...,SCOUR_CRITICAL_113,FUTURE_ADT_114,YEAR_OF_FUTURE_ADT_115,MIN_NAV_CLR_MT_116,FED_AGENCY,SUBMITTED_BY,BRIDGE_CONDITION,LOWEST_RATING,DECK_AREA,FID
0,48,0000000L2015318,1,6,4,00000,0,0,375,0,...,7,200,2035,NaN,Y,68,F,6,169.84,35185702_101560762
1,48,000021521-00101,1,6,0,00000,0,2,71,0,...,8,1,2042,NaN,Y,63,F,6,150.72,29362100_94272880
2,48,000021521-00181,1,6,0,00000,0,2,71,0,...,8,1,2042,NaN,Y,63,G,7,74.40,29362940_94254920
3,48,000021521-TMP20,1,6,1,00000,0,2,71,0,...,8,150,2042,NaN,Y,63,G,7,91.91,29365160_94324020
4,48,000021525-00012,1,6,0,00000,0,2,245,0,...,8,120,2042,NaN,Y,63,P,4,204.30,29413300_94044860


In [43]:
aa['LOWEST_RATING']

0        6
1        6
2        7
3        7
4        4
        ..
56724    7
56725    6
56726    5
56727    6
56728    7
Name: LOWEST_RATING, Length: 56571, dtype: int64

In [44]:
print(aa["LOWEST_RATING"].dtype)

int64


In [45]:
aa["LOWEST_RATING"].describe

<bound method NDFrame.describe of 0        6
1        6
2        7
3        7
4        4
        ..
56724    7
56725    6
56726    5
56727    6
56728    7
Name: LOWEST_RATING, Length: 56571, dtype: int64>

In [46]:
aaf = aa.loc[(aa['LOWEST_RATING']>=0) | (aa['LOWEST_RATING']<10)]

In [47]:
len(aaf)

56571